In [ ]:
# Install selenium itself for the first time
!pip install selenium

In [1]:
# Install libraries dependencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
from bs4 import BeautifulSoup
import json
import os
import re
from selenium.webdriver.firefox.options import Options
import csv

In [7]:
# Retrieve Answers to Questions within CSV file
def retrieve(things):
    options = Options()
    # CHANGE THE LINK TO YOUR FIREFOX PROFILE WHERE CHATGPT IS LOGGED IN + ENABLE COOKIES
    options.profile = ''
    driver = webdriver.Firefox(options=options)
    driver.get("https://chat.openai.com/")
    #Do security Question
    time.sleep(7)
    # To iterate through all the questions listed in the question.csv file
    for i in things:
        topic = i[0]
        # Initial Prompt to CHATGPT, you can change how many questions it generate below
        question = "Here are some sample questions, remember this format: Describe the concept of BLANK in machine learning?, Explain the concept of BLANK in machine learning?, What is the difference between a BLANK and BLANK?, Generate 35 questions similar to the format above this topic in machine learning, add a question mark at the end of each question and only display the questions: " 
        question = question + topic
        # Check if the row is empty in the CSV File
        subdirectory = "Machine Learning GPT Question"
        #check if Subdirectory exists
        if not os.path.exists(subdirectory):
            os.makedirs(subdirectory)
        # The file is saved within a parent directory that is labelled as your topic name with each file labelled as the TOPIC name
        filename = re.sub(r'[<>:"/\\|?*]', '', topic) + '.csv'
        filename = os.path.join(subdirectory, filename)
        # Creates a csv file and iterates between each answer, depositing it in a dictionary and appending it into the jsonl file
        if os.path.exists(filename):
            print(str(filename) + "exists")
        else:
            with open(filename, 'a', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                #select how many times you want to scroll down the web page to retrieve the questions
                for i in range(0):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                    time.sleep(2)
                html_source = driver.page_source
                soup_3 = BeautifulSoup(html_source, 'html.parser')
                # Input question using xpathh
                xpathh = "//textarea[contains(@tabindex,'0')]"
                text_input = driver.find_element(By.XPATH, value=xpathh)
                print(question)
                ActionChains(driver)\
                    .send_keys_to_element(text_input, question)\
                    .perform()
                # Clicking submit
                xpathh = "//button[contains(@class,'absolute bottom-1.5 right-2 rounded-lg border border-black bg-black p-0.5 text-white transition-colors enabled:bg-black disabled:text-gray-400 disabled:opacity-10 dark:border-white dark:bg-white dark:hover:bg-white md:bottom-3 md:right-3')]"
                clickable = driver.find_element(By.XPATH, xpathh)
                ActionChains(driver)\
                    .click(clickable)\
                    .perform()
                time.sleep(70)
                # Find all answer using xpath
                xpathh = "(//div[contains(@class,'markdown prose w-full break-words dark:prose-invert light')])"
                elements = driver.find_elements(By.XPATH, value=xpathh)
                # We wanna break the questions up into its subsets
                element = elements[0]
                inner_html = element.get_attribute('innerHTML')
                soup = BeautifulSoup(inner_html, 'html.parser')
                question_text = soup.get_text(separator=' ', strip=True)
                print(question_text)
                questions = question_text.split("?")
                # Iterate through all the questions
                for question in questions:
                    question = question.strip()
                    if len(question) == 0:
                        continue
                    else:
                        question += "?"
                        things = [topic ,question]
                        writer.writerow(things)
                # Clicking new chat to restart the whole thing again
                xpathh = "//button[contains(@class, 'text-token-text-primary')]"
                clickable = driver.find_element(By.XPATH, xpathh)
                ActionChains(driver)\
                    .click(clickable)\
                    .perform()
                time.sleep(1)
    driver.quit()


    




In [3]:
#Removes empty files due to error links within the subdirectory
def remove_empty_files(subdirectory):
    # Iterate over files in the subdirectory
    for filename in os.listdir(subdirectory):
        filepath = os.path.join(subdirectory, filename)
        # Check if the file is empty
        if os.path.isfile(filepath) and os.path.getsize(filepath) == 0:
            print("Removing empty file:", filename)
            os.remove(filepath)

In [8]:
#Reads the file and retrieves the list of topics
with open("Put the topics you want to scrape here.csv") as topics:
    subdirectory = "Machine Learning GPT Question"
    remove_empty_files(subdirectory)
    next(topics)
    things = []
    for line in topics:
        line = line.rstrip("\n")
        line = line.split(",")
        topic = line[0]
        things.append([topic])
    # Calls the function to find the answer to each question, the directory would be the topic
    retrieve(things)
    # Cleans the subdirectory if there are empty files due to empty question csv files

        

Here are some sample questions, remember this format: Describe the concept of BLANK in machine learning?, Explain the concept of BLANK in machine learning?, What is the difference between a BLANK and BLANK?, Generate 35 questions similar to the format above this topic in machine learning, add a question mark at the end of each question and only display the questions: Machine Learning
What is the concept of overfitting in machine learning? How would you define the term "feature engineering" in machine learning? Describe the process of regularization in machine learning? What role does gradient descent play in machine learning? Explain the concept of cross-validation in machine learning? What is the difference between supervised and unsupervised learning in machine learning? Describe the concept of bias-variance tradeoff in machine learning? How does ensemble learning work in the context of machine learning? What is the purpose of dimensionality reduction in machine learning? Explain the

In [ ]:
# Test run
# This should create open a browser to search for this questions
retrieve([["Machine Learning", "Machine Learning"]])